In [1]:
import ee

# Trigger the authentication flow.
# ee.Authenticate()

ee.Initialize(project='ee-arzaaan789')

In [2]:
import pandas as pd
from tqdm import tqdm

df = pd.read_csv("Erinaceus europaeus.csv", delimiter='\t')
df = df[df["occurrenceStatus"] == "PRESENT"]
df = df[df['year']>=2022]
# df['eventDate'] = df['eventDate'].str.replace('/','')
df['eventDate'] = pd.to_datetime(df['eventDate'], format='%Y-%m-%d', errors='coerce')
df = df[['species', 'decimalLatitude', 'decimalLongitude', 'eventDate']]
df = df.dropna().reset_index(drop=True)

/var/folders/0n/sggn66691kd6g5mtyzpq2hv80000gn/T/ipykernel_84624/233028871.py:4: DtypeWarning: Columns (2,10,16,29,36,37,38,39,40,41,44,45,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Erinaceus europaeus.csv", delimiter='\t')


In [3]:
import ee
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm

# ee.Initialize()

# Your dataframe 'df' must have columns: decimalLongitude, decimalLatitude
# Example: df = pd.read_csv("Erinaceus europaeus.csv", delimiter='\t')

def create_aoi(lon, lat, box_size_km=1):
    """Create approx 1km x 1km square polygon around lon, lat."""
    half_side_deg = box_size_km / 111.32 / 2  # Rough approx degrees per km

    coords = [
        [lon - half_side_deg, lat - half_side_deg],
        [lon + half_side_deg, lat - half_side_deg],
        [lon + half_side_deg, lat + half_side_deg],
        [lon - half_side_deg, lat + half_side_deg],
        [lon - half_side_deg, lat - half_side_deg]
    ]
    return ee.Geometry.Polygon(coords)

def compute_all_indices(feature, start_date, end_date):
    """Compute spectral indices and LST for one EE Feature (with AOI geometry)."""
    aoi = feature.geometry()

    s2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
          .filterBounds(aoi)
          .filterDate(start_date, end_date)
          .sort('CLOUDY_PIXEL_PERCENTAGE')
          .first())
    s2 = ee.Image(s2).clip(aoi)

    blue = s2.select('B2')
    green = s2.select('B3')
    red = s2.select('B4')
    nir = s2.select('B8')
    swir = s2.select('B11')

    L = 0.5  # SAVI constant

    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
    ndwi = green.subtract(nir).divide(green.add(nir)).rename('NDWI')
    ndbi = swir.subtract(nir).divide(swir.add(nir)).rename('NDBI')
    savi = nir.subtract(red).divide(nir.add(red).add(L)).multiply(1 + L).rename('SAVI')
    mndwi = green.subtract(swir).divide(green.add(swir)).rename('MNDWI')
    ndsi = green.subtract(swir).divide(green.add(swir)).rename('NDSI')
    bsi = (red.add(blue).subtract(nir.add(swir))).divide(red.add(blue).add(nir).add(swir)).rename('BSI')
    ui = nir.subtract(swir).divide(nir.add(swir)).rename('UI')

    reducers = ee.Reducer.mean()
    scale_10m = 1000

    ndvi_mean = ndvi.reduceRegion(reducers, aoi, scale_10m).get('NDVI')
    ndwi_mean = ndwi.reduceRegion(reducers, aoi, scale_10m).get('NDWI')
    ndbi_mean = ndbi.reduceRegion(reducers, aoi, scale_10m).get('NDBI')
    savi_mean = savi.reduceRegion(reducers, aoi, scale_10m).get('SAVI')
    mndwi_mean = mndwi.reduceRegion(reducers, aoi, scale_10m).get('MNDWI')
    ndsi_mean = ndsi.reduceRegion(reducers, aoi, scale_10m).get('NDSI')
    bsi_mean = bsi.reduceRegion(reducers, aoi, scale_10m).get('BSI')
    ui_mean = ui.reduceRegion(reducers, aoi, scale_10m).get('UI')

    # MODIS LST dataset
    modis = (ee.ImageCollection("MODIS/061/MOD11A1")
             .filterBounds(aoi)
             .filterDate(start_date, end_date)
             .select('LST_Day_1km'))

    lst_mean_img = modis.mean().multiply(0.02).clip(aoi)

    lst_mean = lst_mean_img.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi,
        scale=1000
    ).get('LST_Day_1km')

    return feature.set({
        'NDVI': ndvi_mean,
        'NDWI': ndwi_mean,
        'NDBI': ndbi_mean,
        'SAVI': savi_mean,
        'MNDWI': mndwi_mean,
        'NDSI': ndsi_mean,
        'BSI': bsi_mean,
        'UI': ui_mean,
        'LST': lst_mean
    })

# Split df into batches
batch_size = 50
batches = [df.iloc[i:i+batch_size] for i in range(0, len(df), batch_size)]

results_list = []

index_names = ['NDVI', 'NDWI', 'NDBI', 'SAVI', 'MNDWI', 'NDSI', 'BSI', 'UI', 'LST']

for batch in tqdm(batches, desc="Processing batches"):
    features = []
    batch_indices = []
    feature_metadata = {}

    for idx, row in batch.iterrows():
        
        aoi = create_aoi(row['decimalLongitude'], row['decimalLatitude'])
        feature = ee.Feature(aoi).set('index', idx)
        features.append(feature)
        batch_indices.append(idx)  # Save the original index
        
        event_date = row['eventDate']
        start_date = (event_date - timedelta(days=15)).strftime('%Y-%m-%d')
        end_date = (event_date + timedelta(days=15)).strftime('%Y-%m-%d')
        feature_metadata[idx] = (start_date, end_date)

    # Create a FeatureCollection from the list of features
    fc = ee.FeatureCollection(features)
    
    # Define wrapper for map to inject per-feature dates
    def map_with_dates(f):
        idx = f.get('index')
        # Use dictionary lookup to get dates for this feature
        date_dict = ee.Dictionary(ee.Dictionary(feature_metadata))
        dates = ee.List(date_dict.get(ee.Number(idx)))
        return compute_all_indices(f, dates.get(0), dates.get(1))

    try:
        result_fc = fc.map(map_with_dates)
        results = result_fc.getInfo()

        rows = []
        for f in results['features']:
            props = f['properties']
            rows.append(props)

    except Exception as e:
        print(f"Error processing batch starting at index {batch.index[0]}: {e}")
        # If there's an error, create placeholder rows with None
        rows = [{'index': i, **{name: None for name in index_names}} for i in batch_indices]

    batch_results_df = pd.DataFrame(rows).sort_values('index').reset_index(drop=True)
    results_list.append(batch_results_df)

# Concatenate all batches and sort by original index
all_results_df = pd.concat(results_list).sort_values('index').reset_index(drop=True)

# Merge with original df
df_final = pd.concat([df.reset_index(drop=True), all_results_df.drop(columns=['index'])], axis=1)

print(df_final.head())


Processing batches: 100%|██████████| 924/924 [2:02:45<00:00,  7.97s/it]  


               species  decimalLatitude  decimalLongitude  eventDate  \
0  Erinaceus europaeus        52.835848         -0.947492 2022-06-19   
1  Erinaceus europaeus        52.818132         -0.977597 2023-05-13   
2  Erinaceus europaeus        52.689385         -0.684702 2022-06-12   
3  Erinaceus europaeus        52.727454         -0.890871 2022-02-25   
4  Erinaceus europaeus        52.710769         -1.039317 2022-07-13   

        BSI         LST     MNDWI      NDBI      NDSI      NDVI      NDWI  \
0 -0.618472  297.235328 -0.406983 -0.266568 -0.406983  0.660731 -0.596190   
1 -0.787202  293.872242 -0.478050 -0.418311 -0.478050  0.824125 -0.746002   
2 -0.743518  294.579695 -0.451649 -0.387726 -0.451649  0.773335 -0.715819   
3 -0.650823  282.179633 -0.510202 -0.118358 -0.510202  0.610773 -0.582729   
4 -0.679126  304.672511 -0.561134 -0.144483 -0.561134  0.641981 -0.652315   

       SAVI        UI  
0  0.991003  0.266568  
1  1.236058  0.418311  
2  1.159877  0.387726  
3  0.916

In [4]:
df_final.to_csv("hedgehog_full_data.csv", index=False)
df = pd.read_csv("hedgehog_full_data.csv")
df['eventDate'] = pd.to_datetime(df['eventDate'], format='%Y-%m-%d', errors='coerce')

df = df.dropna().reset_index(drop=True)

badgers = pd.read_csv("Meles meles.csv", delimiter='\t')
badgers = badgers[badgers["occurrenceStatus"] == "PRESENT"]
badgers = badgers[badgers['year']>=2022]
badgers['eventDate'] = pd.to_datetime(badgers['eventDate'], format='%Y-%m-%d', errors='coerce')
badgers = badgers[['species', 'decimalLatitude', 'decimalLongitude', 'eventDate']]
badgers = badgers.dropna().reset_index(drop=True)

ground_beetles = pd.read_csv("ground_beetles.csv", delimiter='\t')
ground_beetles = ground_beetles[ground_beetles["occurrenceStatus"] == "PRESENT"]
ground_beetles = ground_beetles[ground_beetles['year']>=2022]
ground_beetles['eventDate'] = pd.to_datetime(ground_beetles['eventDate'], format='%Y-%m-%d', errors='coerce')
ground_beetles = ground_beetles[['species', 'decimalLatitude', 'decimalLongitude', 'eventDate']]
ground_beetles = ground_beetles.dropna().reset_index(drop=True)

/var/folders/0n/sggn66691kd6g5mtyzpq2hv80000gn/T/ipykernel_84624/3683829675.py:7: DtypeWarning: Columns (10,16,39,41,45,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  badgers = pd.read_csv("Meles meles.csv", delimiter='\t')
/var/folders/0n/sggn66691kd6g5mtyzpq2hv80000gn/T/ipykernel_84624/3683829675.py:14: DtypeWarning: Columns (2,10,36,38,39,41,45,48) have mixed types. Specify dtype option on import or set low_memory=False.
  ground_beetles = pd.read_csv("ground_beetles.csv", delimiter='\t')


In [5]:
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree
from tqdm import tqdm

# Convert lat/lon to radians for BallTree
hedgehog_coords = np.deg2rad(df[['decimalLatitude', 'decimalLongitude']].values)
badger_coords = np.deg2rad(badgers[['decimalLatitude', 'decimalLongitude']].values)
ground_beetles_coords = np.deg2rad(ground_beetles[['decimalLatitude', 'decimalLongitude']].values)

# Build BallTrees
tree_badger = BallTree(badger_coords, metric='haversine')
tree_ground_beetles = BallTree(ground_beetles_coords, metric='haversine')

# 1 km radius in radians
radius = 1 / 6371.0

# Initialize presence columns
df['badger_presence'] = 0
df['ground_beetles_presence'] = 0

# Iterate through each hedgehog point with tqdm for progress tracking
for i in tqdm(range(len(df))):
    point = hedgehog_coords[i].reshape(1, -1)
    event_month = df.loc[i, 'eventDate'].month
    event_year = df.loc[i, 'eventDate'].year

    # BADGERS
    idxs = tree_badger.query_radius(point, r=radius)[0]
    for j in idxs:
        badger_date = badgers.loc[j, 'eventDate']
        if badger_date.month == event_month and badger_date.year == event_year:
            df.at[i, 'badger_presence'] = 1
            break  # Found at least one match, no need to check further

    # GROUND BEETLES
    idxs = tree_ground_beetles.query_radius(point, r=radius)[0]
    for j in idxs:
        beetle_date = ground_beetles.loc[j, 'eventDate']
        if beetle_date.month == event_month and beetle_date.year == event_year:
            df.at[i, 'ground_beetles_presence'] = 1
            break

100%|██████████| 45936/45936 [00:03<00:00, 11838.65it/s]


In [6]:
import osmnx as ox
from shapely.geometry import Point
import geopandas as gpd
import numpy as np
from tqdm import tqdm

# Construct GeoDataFrame
df['geometry'] = [Point(xy) for xy in zip(df['decimalLongitude'], df['decimalLatitude'])]
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')

# Define spatial tiling: 0.05 x 0.05 degrees
tile_size_lat = 0.009   # ~1 km in latitude degrees
tile_size_lon = 0.0157  # ~1 km in longitude degrees
padding_lat = 0.002  # ~200m latitude padding
padding_lon = 0.003  # ~200-300m longitude padding

gdf['tile_x'] = (gdf['decimalLongitude'] // tile_size_lon).astype(int)
gdf['tile_y'] = (gdf['decimalLatitude'] // tile_size_lat).astype(int)

# Group by tile
grouped = gdf.groupby(['tile_x', 'tile_y'])

# Store results
all_distances = []

for (tile_x, tile_y), group in tqdm(grouped, total=len(grouped)):
    west = tile_x * tile_size_lon - padding_lon
    south = tile_y * tile_size_lat - padding_lat
    east = (tile_x + 1) * tile_size_lon + padding_lon
    north = (tile_y + 1) * tile_size_lat + padding_lat
    bbox = (west, south, east, north)


    try:
        G = ox.graph_from_bbox(bbox, network_type='drive_service')
        if len(G.nodes) == 0:
            print(f"[EMPTY GRAPH] bbox: {bbox}, skipping...")
            all_distances.extend([np.nan] * len(group))
            continue

        G_proj = ox.project_graph(G)
        nodes_proj, edges_proj = ox.graph_to_gdfs(G_proj)
        points_proj = group.geometry.to_crs(nodes_proj.crs)

        for point_proj in points_proj:
            try:
                u, v, k = ox.distance.nearest_edges(G_proj, [point_proj.x], [point_proj.y])[0]
                edge_geom = edges_proj.loc[(u, v, k)]['geometry']
                distance = point_proj.distance(edge_geom)
                all_distances.append(distance)
            except Exception as e:
                print(f"  [Point ERROR] {e}")
                all_distances.append(np.nan)

    except Exception as e:
        print(f"[TILE ERROR] {e} — bbox: {bbox}")
        all_distances.extend([np.nan] * len(group))


# Store back in original DataFrame order
gdf['distance_to_road'] = all_distances

  0%|          | 2/17350 [00:00<44:55,  6.44it/s]

[TILE ERROR] No data elements in server response. Check query location/filters and log. — bbox: (np.float64(-7.8687), np.float64(54.483999999999995), np.float64(-7.8469999999999995), np.float64(54.497))


  0%|          | 60/17350 [00:23<1:26:42,  3.32it/s] 

[TILE ERROR] Graph contains no edges. — bbox: (np.float64(-7.2406999999999995), np.float64(54.74499999999999), np.float64(-7.218999999999999), np.float64(54.757999999999996))


  1%|▏         | 233/17350 [01:48<11:18:35,  2.38s/it]

...

 99%|█████████▉| 17242/17350 [13:21:40<05:58,  3.32s/it]   

[TILE ERROR] Found no graph nodes within the requested polygon. — bbox: (np.float64(1.4727999999999999), np.float64(52.50399999999999), np.float64(1.4944999999999997), np.float64(52.516999999999996))


100%|█████████▉| 17295/17350 [13:25:09<03:33,  3.89s/it]

[TILE ERROR] Found no graph nodes within the requested polygon. — bbox: (np.float64(1.6298), np.float64(52.49499999999999), np.float64(1.6514999999999997), np.float64(52.507999999999996))


100%|██████████| 17350/17350 [13:32:11<00:00,  2.81s/it]


In [7]:
gdf['near_road'] = np.where(gdf['distance_to_road'] <= 500, 1, 0)        

In [8]:
gdf=gdf.drop(['tile_x', 'tile_y', 'geometry', 'distance_to_road'], axis=1)

In [9]:
from pyproj import Transformer
from rasterio.windows import Window
import rasterio
from collections import Counter
import numpy as np


land_cover_map = {
    1: "Deciduous woodland",
    2: "Coniferous woodland",
    3: "Arable",
    4: "Improved grassland",
    5: "Neutral grassland",
    6: "Calcareous grassland",
    7: "Acid grassland",
    8: "Fen",
    9: "Heather",
    10: "Heather grassland",
    11: "Bog",
    12: "Inland rock",
    13: "Saltwater",
    14: "Freshwater",
    15: "Supralittoral rock",
    16: "Supralittoral sediment",
    17: "Littoral rock",
    18: "Littoral sediment",
    19: "Saltmarsh",
    20: "Urban",
    21: "Suburban"
}

# Batch coordinate transformation
transformer_ni = Transformer.from_crs("EPSG:4326", "EPSG:29903", always_xy=True)
transformer_gb = Transformer.from_crs("EPSG:4326", "EPSG:27700", always_xy=True)

coords = list(zip(gdf['decimalLongitude'], gdf['decimalLatitude']))
gdf['easting_ni'], gdf['northing_ni'] = zip(*transformer_ni.itransform(coords))
gdf['easting_gb'], gdf['northing_gb'] = zip(*transformer_gb.itransform(coords))

# Raster processing optimization
gb_raster = 'gblcm2023_10m.tif'
n_ireland_raster = 'nilcm2023_10m.tif'


def get_land_cover_class_1km(row):
    try:
        window_size_pixels = 100  # 1 km / 10m resolution

        # Open GB raster and read 1km x 1km window
        with rasterio.open(gb_raster) as src:
            row_idx, col_idx = src.index(row['easting_gb'], row['northing_gb'])

            # Calculate window bounds, making sure not to go out of raster bounds
            row_start = max(row_idx - window_size_pixels // 2, 0)
            col_start = max(col_idx - window_size_pixels // 2, 0)

            # Adjust window size if near edges
            height = min(window_size_pixels, src.height - row_start)
            width = min(window_size_pixels, src.width - col_start)

            window = Window(col_start, row_start, width, height)
            data = src.read(1, window=window)

            # If all zero (or no data), fallback to NI raster
            if np.all(data == 0):
                with rasterio.open(n_ireland_raster) as src_ni:
                    row_idx_ni, col_idx_ni = src_ni.index(row['easting_ni'], row['northing_ni'])

                    row_start_ni = max(row_idx_ni - window_size_pixels // 2, 0)
                    col_start_ni = max(col_idx_ni - window_size_pixels // 2, 0)

                    height_ni = min(window_size_pixels, src_ni.height - row_start_ni)
                    width_ni = min(window_size_pixels, src_ni.width - col_start_ni)

                    window_ni = Window(col_start_ni, row_start_ni, width_ni, height_ni)
                    data_ni = src_ni.read(1, window=window_ni)

                    data = data_ni

            # Find the most common class (mode) ignoring zeros (assuming 0 means no data)
            unique, counts = np.unique(data[data != 0], return_counts=True)
            if len(counts) == 0:
                return "Unknown"

            mode_class = unique[np.argmax(counts)]
            return land_cover_map.get(mode_class, "Unknown")

    except Exception as e:
        print(f"Error processing row: {e}")
        return "Unknown"


tqdm.pandas()
gdf['Land_cover'] = gdf.progress_apply(get_land_cover_class_1km, axis=1)


 90%|████████▉ | 41138/45936 [1:06:00<07:21, 10.87it/s]

Error processing row: Number of columns or rows must be non-negative


 96%|█████████▋| 44317/45936 [1:11:15<01:24, 19.19it/s]

Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative


 97%|█████████▋| 44350/45936 [1:11:18<02:59,  8.86it/s]

Error processing row: Number of columns or rows must be non-negative


 97%|█████████▋| 44358/45936 [1:11:19<03:07,  8.42it/s]

Error processing row: Number of columns or rows must be non-negative


100%|██████████| 45936/45936 [1:14:07<00:00, 10.33it/s]


In [10]:
gdf = gdf.dropna()
gdf=gdf.drop(['easting_ni', 'northing_ni', 'easting_gb', 'northing_gb'],axis=1)
# Remove where Land_cover is Unknown
gdf = gdf[gdf['Land_cover'] != "Unknown"].reset_index(drop=True)
gdf.to_csv("hedgehog_final_data.csv", index=False)